In [1]:
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/bwhpc/common/devel/cuda/11.8

env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/bwhpc/common/devel/cuda/11.8


In [2]:
import tensorflow_datasets as tfds
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import json
from collections import defaultdict
from skimage import io
from skimage.color import rgb2gray
#from skimage.color import rgb2gray, rgb2grey
from scipy.ndimage.filters import gaussian_filter
import imageio
import math

2023-01-09 16:20:08.601253: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 16:20:14.857821: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-09 16:20:34.701952: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-01-09 16:20:34.711428: W tensorflow/compil

In [3]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)

2023-01-09 16:21:40.676292: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 16:21:43.296337: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 337 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:15:00.0, compute capability: 7.0


Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2023-01-09 16:21:43.314554: I tensorflow/core/common_runtime/placer.cc:114] input: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:21:43.314593: I tensorflow/core/common_runtime/placer.cc:114] _EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:21:43.314602: I tensorflow/core/common_runtime/placer.cc:114] output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:21:43.329714: I tensorflow/core/common_runtime/placer.cc:114] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:21:43.329744: I tensorflow/core/common_runtime/placer.cc:114] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:21:43.331564: I tensorflow/core/common_runtime/placer.cc:114] resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:21:43.331584: I tensorflow/core/common_runtime/placer.cc:114] value: (_Arg): /job:localhost/replica:0/task:0/device:G

In [5]:
path_to_datasets = './saved_datasets/'

In [6]:
with strategy.scope():
    train_0_14 = tf.data.Dataset.load(os.path.join(path_to_datasets, "train", "0-14_ds_without_batching"))
    train_14 = tf.data.Dataset.load(os.path.join(path_to_datasets, "train", "14_ds_without_batching"))
    train_15 = tf.data.Dataset.load(os.path.join(path_to_datasets, "train", "15_ds_without_batching"))

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LoadDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LoadDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LoadDataset in device /job:localhost/replica:0/task:0/device:CPU:0


2023-01-09 16:22:08.481242: I tensorflow/core/common_runtime/placer.cc:114] input: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:22:08.481271: I tensorflow/core/common_runtime/placer.cc:114] _EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:22:08.481277: I tensorflow/core/common_runtime/placer.cc:114] output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:22:08.486684: I tensorflow/core/common_runtime/placer.cc:114] path: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:22:08.486710: I tensorflow/core/common_runtime/placer.cc:114] LoadDataset: (LoadDataset): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:22:08.486716: I tensorflow/core/common_runtime/placer.cc:114] handle_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:22:08.523477: I tensorflow/core/common_runtime/placer.cc:114] path: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
20

In [7]:
train_14_prepared = train_14.take(10000)
train_15_prepared = train_15.take(10000)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0


2023-01-09 16:22:21.908614: I tensorflow/core/common_runtime/placer.cc:114] input: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:22:21.908645: I tensorflow/core/common_runtime/placer.cc:114] _EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:22:21.908651: I tensorflow/core/common_runtime/placer.cc:114] output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-01-09 16:22:21.910841: I tensorflow/core/common_runtime/placer.cc:114] input__dataset: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:22:21.910859: I tensorflow/core/common_runtime/placer.cc:114] count: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:22:21.910866: I tensorflow/core/common_runtime/placer.cc:114] TakeDataset: (TakeDataset): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:22:21.910872: I tensorflow/core/common_runtime/placer.cc:114] handle_RetVal: (_Retval): /job:localhost/replica:0/task:0/devi

In [14]:
len(train_15_prepared)

Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0


10000

In [13]:
len(train_15)

Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0


130860

In [15]:
len(train_0_14)

Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0


60175

In [11]:
res = train_0_14.concatenate(train_15_prepared).concatenate(train_14_prepared)

Executing op ConcatenateDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ConcatenateDataset in device /job:localhost/replica:0/task:0/device:CPU:0


2023-01-09 16:23:16.393769: I tensorflow/core/common_runtime/placer.cc:114] input__dataset: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:23:16.393797: I tensorflow/core/common_runtime/placer.cc:114] another__dataset: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:23:16.393805: I tensorflow/core/common_runtime/placer.cc:114] ConcatenateDataset: (ConcatenateDataset): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:23:16.393811: I tensorflow/core/common_runtime/placer.cc:114] handle_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0


In [12]:
tf.data.Dataset.save(res, "./saved_datasets/train_ds_without_batching")

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op SaveDataset in device /job:localhost/replica:0/task:0/device:CPU:0


2023-01-09 16:23:55.652204: I tensorflow/core/common_runtime/placer.cc:114] input__dataset: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:23:55.652238: I tensorflow/core/common_runtime/placer.cc:114] path: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:23:55.652250: I tensorflow/core/common_runtime/placer.cc:114] SaveDataset: (SaveDataset): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:23:55.704982: I tensorflow/core/common_runtime/placer.cc:114] args_0: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:23:55.705003: I tensorflow/core/common_runtime/placer.cc:114] args_1: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:23:56.215650: I tensorflow/core/common_runtime/placer.cc:114] args_0: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:23:56.215680: I tensorflow/core/common_runtime/placer.cc:114] ParallelInterleaveDatasetV4: (ParallelInterleaveDatasetV4): /job:localhost/replica:0

input: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
_EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
value: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
AssignVariableOp: (AssignVariableOp): /job:localhost/replica:0/task:0/device:GPU:0
input: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
_EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
path: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
LoadDataset: (LoadDataset): /job:localhost/replica:0/task:0/device:CPU:0
handle_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
path: (_Arg): /job:localhost/replica:

2023-01-09 16:24:35.893342: I tensorflow/core/common_runtime/placer.cc:114] args_0: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:24:35.893375: I tensorflow/core/common_runtime/placer.cc:114] Identity: (Identity): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:24:35.893381: I tensorflow/core/common_runtime/placer.cc:114] FakeSink0: (Identity): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:24:35.893386: I tensorflow/core/common_runtime/placer.cc:114] identity_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:24:41.676006: I tensorflow/core/common_runtime/placer.cc:114] args_0: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:24:41.676045: I tensorflow/core/common_runtime/placer.cc:114] ParallelInterleaveDatasetV4: (ParallelInterleaveDatasetV4): /job:localhost/replica:0/task:0/device:CPU:0
2023-01-09 16:24:41.676052: I tensorflow/core/common_runtime/placer.cc:114] Identity: (Identity): /job:localh